# Introduction

## Major imports

In [25]:
import pandas as pd
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

In [26]:
df = pd.read_csv(filepath_or_buffer='ncses_table_srv_data_NSCG_2024-11-11T19_30_54Z\\ncses_table_raw_data_NSCG_2024-11-11T19_30_54Z.csv',thousands=',')
df.dropna(how='all',axis='index')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 6 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Year                                                312 non-null    int64 
 1   Labor force status                                  312 non-null    object
 2   Race (2003-2021)                                    312 non-null    object
 3   Gender                                              312 non-null    object
 4   Field of maj - most recent deg (major) (2003-2021)  312 non-null    object
 5   Weighted Count                                      312 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 14.8+ KB


In [27]:
#Saving old frame just in case
df_old = df
races = df['Race (2003-2021)'].unique()
df = df.replace(to_replace=races, value= list(range(0, len(races))))
df

C:\Users\pauli\AppData\Local\Temp\ipykernel_11056\2632857430.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(to_replace=races, value= list(range(0, len(races))))


,Year,Labor force status,Race (2003-2021),Gender,Field of maj - most recent deg (major) (2003-2021),Weighted Count
0,2021,Employed,0,Female,Computer and mathematical sciences,280222
1,2021,Employed,0,Female,"Biological, agricultural and environmental lif...",197031
2,2021,Employed,0,Female,Physical and related sciences,45603
3,2021,Employed,0,Female,Social and related sciences,259090
4,2021,Employed,0,Female,Engineering,213556
...,...,...,...,...,...,...
307,2019,Unemployed,5,Male,Physical and related sciences,1573
308,2019,Unemployed,5,Male,Social and related sciences,1359
309,2019,Unemployed,5,Male,Engineering,686
310,2019,Unemployed,5,Male,S&E-Related Fields,2679


In [28]:
def toNum(dataf,column):
    lst = dataf[column].unique()
    dataf = dataf.replace(to_replace = lst, value= list(range(0,len(lst))))
    return dataf

In [29]:
columns = df.columns
columns = columns.drop(labels='Weighted Count',)
for x in columns:
    df = toNum(df,x)
columns

C:\Users\pauli\AppData\Local\Temp\ipykernel_11056\1756994110.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataf = dataf.replace(to_replace = lst, value= list(range(0,len(lst))))


Index(['Year', 'Labor force status', 'Race (2003-2021)', 'Gender',
       'Field of maj - most recent deg (major) (2003-2021)'],
      dtype='object')

In [30]:
X = df.drop(columns = 'Labor force status')
y = df['Labor force status']


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=746, test_size=0.3)

In [32]:
model = linear_model.LogisticRegression()
model.fit(X_train,y_train)

c:\Users\pauli\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()